In [2]:
import json
import csv
import numpy as np

from collections import defaultdict
from glob import glob

jsonFileName = './inputJson/genWin.json'

f = open(jsonFileName)
inputJson = json.load(f)
dataPathBase = inputJson['dataPathBase']
dataType = inputJson['dataType']
cudaID = 5
f.close()

print("Datatype:", dataType)
if dataType == 'survey':
    fs = 1000 # 1 kHz
    dSampFactor = 2
    nSubC = 30
    nRX = 3
    
    activities = ['bed', 'fall', 'run', 'sitdown', 'standup', 'walk']

padLen = 9
if dataType == "survey":
    dataPath = dataPathBase + "HAR_survey/"
dataDict = {activity:[] for activity in activities}
for activity in activities:
    dataDict[activity] = defaultdict(list)
windDataPath = dataPath + "noWin_pad" + str(padLen) + "/"

padLen = 9
print(dataPath)
noWindDataPath = dataPath + "noWin_pad" + str(padLen) + "/"
for activity in activities:
    if dataType == "survey":
        fileNameList = glob(dataPath + "/input_" + activity + "*.csv")
        outputXXFileName = noWindDataPath + "xx_" + activity

        print(activity, len(fileNameList))

        # xx = np.empty([0, (winLen//dSampFactor + padSize), nSubC*nRX], float)
    dataList = []

    for fileIndex, fileName in enumerate(fileNameList):
        data = np.array([[float(elm) for elm in v] for v in csv.reader(open(fileName, 'r'))])
        annotFileName = fileName.replace('input', 'annotation')
        annot = np.array([[str(elm) for elm in v] for v in csv.reader(open(annotFileName, 'r'))])
        annotDownSamp = annot[::dSampFactor]

        indActs = np.where(annot[:, 0] == activity)[0]
        indPad = np.arange(indActs[0]-padLen*dSampFactor, indActs[0], dSampFactor)
        indActswithPad = np.concatenate((indPad, indActs))
        print(indActs.shape, indActswithPad.shape)
        # print(indPad, indActs[:5], indActswithPad[:15])
        
        dataActs = data[indActswithPad, 1:1+nSubC*nRX]
        dataList.append(dataActs)

    np.save(outputXXFileName, np.array(dataList, dtype=object), allow_pickle=True)
    # with open(outputXXFileName, "w+") as f:
    #     pickle.dump(dataList, f)

Datatype: survey
